IMPORTS & PREPRODUCTION

In [1]:
import time

In [2]:
import os
import tensorflow as tf

#os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

if tf.test.gpu_device_name():
    print('GPU found')
else:
    print("No GPU found")

GPU found


In [3]:
import tensorflow as tf
from tensorflow import keras as keras
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

In [4]:
import wandb
wandb.login()
wandb.init(project="LeNet_5_test", entity="hyunseokerikjung")

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: hyunseokerikjung. Use `wandb login --relogin` to force relogin


In [5]:
from wandb.keras import WandbCallback

Data Processing

In [6]:
data = pd.read_csv('emnist-byclass-mapping.txt', sep=" ", header=None)
data.columns = ['Target','ASCII']

col=[]
for i in data.ASCII.tolist():
    col.append(chr(i))

data['ASCII_Target'] = pd.Series(col)

In [7]:
train_df = pd.read_csv('emnist-byclass-train.csv/emnist-byclass-train.csv')
train_df.rename(columns={'35':'Target'}, inplace=True)

# train_df=pd.merge(train_df, data, how='left', on='Target')
# train_df=train_df.drop(['Target','ASCII'],axis=1)
# train_df.rename(columns={'ASCII_Target':'Target'}, inplace=True)

train_y = np.array(train_df.pop('Target'))
train_x = train_df

train_x = train_x / 255.0
train_x = train_x.values.reshape([-1, 28, 28, 1])
train_y = keras.utils.to_categorical(train_y)

train_x, val_x, train_y, val_y = train_test_split(train_x, train_y, test_size=0.2, random_state=42)

In [8]:
test_df = pd.read_csv('emnist-byclass-test.csv/emnist-byclass-test.csv')
test_df.rename(columns={'18':'Target'}, inplace=True)

# test_df=pd.merge(test_df, data, how='left', on='Target')
# test_df=test_df.drop(['Target','ASCII'],axis=1)
# test_df.rename(columns={'ASCII_Target':'Target'}, inplace=True)

test_y = np.array(test_df.pop('Target'))
test_x = test_df

test_x = test_x / 255.0
test_x = test_x.values.reshape([-1, 28, 28, 1])
test_y = keras.utils.to_categorical(test_y)

In [9]:
hand_df = pd.read_csv('our_handmade_dataset.csv')
hand_df.rename(columns={'0':'Target'}, inplace=True)

# hand_df=pd.merge(hand_df, data, how='left', on='Target')
# hand_df=hand_df.drop(['Target','ASCII'],axis=1)
# hand_df.rename(columns={'ASCII_Target':'Target'}, inplace=True)

hand_y = np.array(hand_df.pop('Target'))
hand_x = hand_df

hand_x = hand_x / 255.0
hand_x = hand_x.values.reshape([-1, 28, 28, 1])
hand_y = keras.utils.to_categorical(hand_y)

MODELS

Original LeNet_5

In [20]:
lenet_5_model = keras.models.Sequential([
    keras.layers.Conv2D(6, kernel_size=5, strides=1,  activation='tanh', input_shape=train_x[0].shape, padding='same'),
    keras.layers.AveragePooling2D(),
    keras.layers.Conv2D(16, kernel_size=5, strides=1, activation='tanh', padding='valid'),
    keras.layers.AveragePooling2D(),
    keras.layers.Conv2D(120, kernel_size=5, strides=1, activation='tanh', padding='valid'),
    keras.layers.Flatten(),   
    keras.layers.Dense(84, activation='tanh'),
    keras.layers.Dense(62, activation='softmax')
])
lenet_5_model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 28, 28, 6)         156       
                                                                 
 average_pooling2d_2 (Averag  (None, 14, 14, 6)        0         
 ePooling2D)                                                     
                                                                 
 conv2d_7 (Conv2D)           (None, 10, 10, 16)        2416      
                                                                 
 average_pooling2d_3 (Averag  (None, 5, 5, 16)         0         
 ePooling2D)                                                     
                                                                 
 conv2d_8 (Conv2D)           (None, 1, 1, 120)         48120     
                                                                 
 flatten_2 (Flatten)         (None, 120)              

In [ ]:
lenet_5_model.compile(optimizer='adam', loss=keras.losses.categorical_crossentropy, metrics=['accuracy'])

early_stopping_cb = keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)
model_checkpoint_cb = keras.callbacks.ModelCheckpoint("my_lenet5_emnist_model.h5", save_best_only=True)

lenet_5_model.fit(train_x, train_y, epochs=100, validation_data=(val_x, val_y), callbacks=[early_stopping_cb])

lenet_5_model.evaluate(test_x, test_y)

LeNet_5_SeLU

SeLU performs better(speed & accuracy)

In [10]:
keras.backend.clear_session()

lenet_5_selu_model = keras.models.Sequential([
    keras.layers.Conv2D(6, kernel_size=5, strides=1,  activation="selu", kernel_initializer="lecun_normal", input_shape=train_x[0].shape, padding='same'),
    keras.layers.AveragePooling2D(),
    keras.layers.Conv2D(16, kernel_size=5, strides=1, activation="selu", kernel_initializer="lecun_normal", padding='valid'),
    keras.layers.AveragePooling2D(),
    keras.layers.Conv2D(120, kernel_size=5, strides=1, activation="selu", kernel_initializer="lecun_normal", padding='valid'),
    keras.layers.Flatten(),   
    keras.layers.Dense(84, activation="selu", kernel_initializer="lecun_normal"),
    keras.layers.Dense(62, activation='softmax')
])

lenet_5_selu_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 28, 28, 6)         156       
                                                                 
 average_pooling2d (AverageP  (None, 14, 14, 6)        0         
 ooling2D)                                                       
                                                                 
 conv2d_1 (Conv2D)           (None, 10, 10, 16)        2416      
                                                                 
 average_pooling2d_1 (Averag  (None, 5, 5, 16)         0         
 ePooling2D)                                                     
                                                                 
 conv2d_2 (Conv2D)           (None, 1, 1, 120)         48120     
                                                                 
 flatten (Flatten)           (None, 120)               0

In [11]:
lenet_5_selu_model.compile(optimizer='adam', loss=keras.losses.categorical_crossentropy, metrics=['accuracy'])

early_stopping_cb = keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)
model_checkpoint_cb = keras.callbacks.ModelCheckpoint("my_lenet5_emnist_model.h5", save_best_only=True)

lenet_5_selu_model.fit(train_x, train_y, epochs=10, validation_data=(val_x, val_y), callbacks=[early_stopping_cb])

lenet_5_selu_model.evaluate(test_x, test_y)

Epoch 1/10
17449/17449 [==============================] - 63s 3ms/step - loss: 0.5903 - accuracy: 0.8034 - val_loss: 0.4675 - val_accuracy: 0.8360
Epoch 2/10
17449/17449 [==============================] - 56s 3ms/step - loss: 0.4577 - accuracy: 0.8374 - val_loss: 0.4607 - val_accuracy: 0.8349
Epoch 3/10
17449/17449 [==============================] - 56s 3ms/step - loss: 0.4352 - accuracy: 0.8441 - val_loss: 0.4475 - val_accuracy: 0.8412
Epoch 4/10
17449/17449 [==============================] - 57s 3ms/step - loss: 0.4231 - accuracy: 0.8470 - val_loss: 0.4274 - val_accuracy: 0.8461
Epoch 5/10
17449/17449 [==============================] - 55s 3ms/step - loss: 0.4146 - accuracy: 0.8495 - val_loss: 0.4431 - val_accuracy: 0.8433
Epoch 6/10
17449/17449 [==============================] - 55s 3ms/step - loss: 0.4087 - accuracy: 0.8510 - val_loss: 0.4219 - val_accuracy: 0.8510
Epoch 7/10
17449/17449 [==============================] - 58s 3ms/step - loss: 0.4045 - accuracy: 0.8516 - val_loss: 0

[0.41336914896965027, 0.8514984250068665]

He Yanmei SeLU Lenet_5 model

Lowers amount of parameters = lower inference time, less stable model

In [ ]:
keras.backend.clear_session()

lenet_5_improved_model = keras.models.Sequential([
    keras.layers.Conv2D(32, kernel_size=5, strides=2, activation="selu", kernel_initializer="lecun_normal", input_shape=train_x[0].shape, padding='same'),
    keras.layers.Conv2D(64, kernel_size=3, strides=2, activation="selu", kernel_initializer="lecun_normal", padding='valid'),
    keras.layers.Conv2D(32, kernel_size=1, strides=1, activation="selu", kernel_initializer="lecun_normal", padding='valid'),
    keras.layers.GlobalAveragePooling2D(),
    keras.layers.Flatten(),
    keras.layers.Dense(62, activation='softmax')
])

In [ ]:
lenet_5_improved_model.compile(optimizer='adam', loss=keras.losses.categorical_crossentropy, metrics=['accuracy'])

early_stopping_cb = keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)
model_checkpoint_cb = keras.callbacks.ModelCheckpoint("my_lenet5_emnist_model.h5", save_best_only=True)

lenet_5_improved_model.fit(train_x, train_y, epochs=100, validation_data=(val_x, val_y), callbacks=[early_stopping_cb])

lenet_5_improved_model.evaluate(test_x, test_y)

Build #20220515

87.09% accuracy

In [14]:
keras.backend.clear_session()

my_lenet_5_selu_model = keras.models.Sequential([
    keras.layers.Conv2D(32, kernel_size=5, strides=1,  activation="selu", kernel_initializer="lecun_normal", input_shape=train_x[0].shape, padding='same'),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(64, kernel_size=5, strides=1, activation="selu", kernel_initializer="lecun_normal", padding='valid'),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(32, kernel_size=3, strides=1, activation="selu", kernel_initializer="lecun_normal", padding='valid'),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.3),
    keras.layers.GlobalAveragePooling2D(),
    keras.layers.Flatten(),
    keras.layers.Dense(62, activation='softmax')
])

my_lenet_5_selu_model.summary()

Epoch 1/100
17449/17449 [==============================] - 74s 4ms/step - loss: 0.7140 - accuracy: 0.7826 - val_loss: 0.5925 - val_accuracy: 0.8015
Epoch 2/100
17449/17449 [==============================] - 67s 4ms/step - loss: 0.4887 - accuracy: 0.8336 - val_loss: 0.4914 - val_accuracy: 0.8313
Epoch 3/100
17449/17449 [==============================] - 69s 4ms/step - loss: 0.4593 - accuracy: 0.8417 - val_loss: 0.4470 - val_accuracy: 0.8422
Epoch 4/100
17449/17449 [==============================] - 68s 4ms/step - loss: 0.4448 - accuracy: 0.8452 - val_loss: 0.4483 - val_accuracy: 0.8451
Epoch 5/100
17449/17449 [==============================] - 64s 4ms/step - loss: 0.4345 - accuracy: 0.8479 - val_loss: 0.4292 - val_accuracy: 0.8479
Epoch 6/100
17449/17449 [==============================] - 64s 4ms/step - loss: 0.4279 - accuracy: 0.8499 - val_loss: 0.4252 - val_accuracy: 0.8513
Epoch 7/100
17449/17449 [==============================] - 63s 4ms/step - loss: 0.4225 - accuracy: 0.8515 - val_

[3.939462184906006, 0.27632108330726624]

In [ ]:
my_lenet_5_selu_model.compile(optimizer='adam', loss=keras.losses.categorical_crossentropy, metrics=['accuracy'])

early_stopping_cb = keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)
model_checkpoint_cb = keras.callbacks.ModelCheckpoint("my_lenet5_emnist_model.h5", save_best_only=True)
  
my_lenet_5_selu_model.fit(train_x, train_y, epochs=100, validation_data=(val_x, val_y), callbacks=[early_stopping_cb, model_checkpoint_cb])

my_lenet_5_selu_model.evaluate(test_x, test_y)

In [ ]:
my_lenet_5_selu_model.evaluate(hand_x, hand_y)

In [ ]:
my_lenet_5_selu_model.save("")

Build #20220516

86.74% accuracy

In [31]:
keras.backend.clear_session()

my_lenet_5_selu_model = keras.models.Sequential([
    keras.layers.Conv2D(32, kernel_size=5, strides=1,  activation="selu", kernel_initializer="lecun_normal", input_shape=train_x[0].shape, padding='same'),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(64, kernel_size=3, strides=1, activation="selu", kernel_initializer="lecun_normal", padding='valid'),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(32, kernel_size=3, strides=1, activation="selu", kernel_initializer="lecun_normal", padding='valid'),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.3),
    keras.layers.GlobalAveragePooling2D(),
    keras.layers.Flatten(),
    keras.layers.Dense(62, activation='softmax')
])

my_lenet_5_selu_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 28, 28, 32)        832       
                                                                 
 batch_normalization (BatchN  (None, 28, 28, 32)       128       
 ormalization)                                                   
                                                                 
 conv2d_1 (Conv2D)           (None, 26, 26, 64)        18496     
                                                                 
 batch_normalization_1 (Batc  (None, 26, 26, 64)       256       
 hNormalization)                                                 
                                                                 
 conv2d_2 (Conv2D)           (None, 24, 24, 32)        18464     
                                                                 
 batch_normalization_2 (Batc  (None, 24, 24, 32)       1

In [ ]:
my_lenet_5_selu_model.compile(optimizer='adam', loss=keras.losses.categorical_crossentropy, metrics=['accuracy'])

early_stopping_cb = keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)
model_checkpoint_cb = keras.callbacks.ModelCheckpoint("my_lenet5_emnist_model.h5", save_best_only=True)
  
my_lenet_5_selu_model.fit(train_x, train_y, epochs=100, validation_data=(val_x, val_y), callbacks=[early_stopping_cb, model_checkpoint_cb])

my_lenet_5_selu_model.evaluate(test_x, test_y)

In [ ]:
my_lenet_5_selu_model.evaluate(hand_x, hand_y)

Build #20220517

testing wandb

In [33]:
keras.backend.clear_session()

lenet_5_selu_model = keras.models.Sequential([
    keras.layers.Conv2D(6, kernel_size=5, strides=1,  activation="selu", kernel_initializer="lecun_normal", input_shape=train_x[0].shape, padding='same'),
    keras.layers.AveragePooling2D(),
    keras.layers.Conv2D(16, kernel_size=5, strides=1, activation="selu", kernel_initializer="lecun_normal", padding='valid'),
    keras.layers.AveragePooling2D(),
    keras.layers.Conv2D(120, kernel_size=5, strides=1, activation="selu", kernel_initializer="lecun_normal", padding='valid'),
    keras.layers.Flatten(),   
    keras.layers.Dense(84, activation="selu", kernel_initializer="lecun_normal"),
    keras.layers.Dense(62, activation='softmax')
])

lenet_5_selu_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 28, 28, 6)         156       
                                                                 
 average_pooling2d (AverageP  (None, 14, 14, 6)        0         
 ooling2D)                                                       
                                                                 
 conv2d_1 (Conv2D)           (None, 10, 10, 16)        2416      
                                                                 
 average_pooling2d_1 (Averag  (None, 5, 5, 16)         0         
 ePooling2D)                                                     
                                                                 
 conv2d_2 (Conv2D)           (None, 1, 1, 120)         48120     
                                                                 
 flatten (Flatten)           (None, 120)               0

In [34]:
lenet_5_selu_model.compile(optimizer='adam', loss=keras.losses.categorical_crossentropy, metrics=['accuracy'])

early_stopping_cb = keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)

lenet_5_selu_model.fit(train_x, train_y, epochs=100, validation_data=(val_x, val_y), callbacks=[early_stopping_cb, WandbCallback()])

lenet_5_selu_model.evaluate(test_x, test_y)

wandb: WARNING The save_model argument by default saves the model in the HDF5 format that cannot save custom objects like subclassed models and custom layers. This behavior will be deprecated in a future release in favor of the SavedModel format. Meanwhile, the HDF5 model is saved as W&B files and the SavedModel as W&B Artifacts.


Epoch 1/100
17449/17449 [==============================] - 56s 3ms/step - loss: 0.5971 - accuracy: 0.8012 - val_loss: 0.4906 - val_accuracy: 0.8291 - _timestamp: 1652767874.0000 - _runtime: 2725.0000
Epoch 2/100
17449/17449 [==============================] - 55s 3ms/step - loss: 0.4583 - accuracy: 0.8372 - val_loss: 0.4541 - val_accuracy: 0.8366 - _timestamp: 1652767929.0000 - _runtime: 2780.0000
Epoch 3/100
17449/17449 [==============================] - 57s 3ms/step - loss: 0.4361 - accuracy: 0.8434 - val_loss: 0.4447 - val_accuracy: 0.8411 - _timestamp: 1652767986.0000 - _runtime: 2837.0000
Epoch 4/100
17449/17449 [==============================] - 55s 3ms/step - loss: 0.4242 - accuracy: 0.8466 - val_loss: 0.4344 - val_accuracy: 0.8452 - _timestamp: 1652768041.0000 - _runtime: 2892.0000
Epoch 5/100
17449/17449 [==============================] - 56s 3ms/step - loss: 0.4164 - accuracy: 0.8489 - val_loss: 0.4324 - val_accuracy: 0.8459 - _timestamp: 1652768097.0000 - _runtime: 2948.0000


[0.4185050129890442, 0.8534842729568481]

In [10]:
def sweep_train(config_defaults=None):
    # Set default values
    config_defaults = {
      'batch_size': 64,
      'optimizer' : 'adam',
      'learning_rate' : 0.01,
      'n_first_node' : 6,
      'n_first_kernel_size' : 5,
      'n_second_node' : 16,
      'n_second_kernel_size' : 5,
      'n_third_node' : 120,
      'n_third_kernel_size' : 5,
    }
    # Initialize wandb with a sample project name
    wandb.init(config=config_defaults)  # this gets over-written in the Sweep

    # initialize model
    model = keras.models.Sequential()
    model.add(keras.layers.Conv2D(wandb.config.n_first_node, kernel_size=wandb.config.n_first_kernel_size, strides=1,  activation="selu", kernel_initializer="lecun_normal", input_shape=train_x[0].shape, padding='same'))
    model.add(keras.layers.AveragePooling2D())
    model.add(keras.layers.Conv2D(wandb.config.n_second_node, kernel_size=wandb.config.n_second_kernel_size, strides=1, activation="selu", kernel_initializer="lecun_normal", padding='valid'))
    model.add(keras.layers.AveragePooling2D())
    model.add(keras.layers.Conv2D(wandb.config.n_third_node, kernel_size=wandb.config.n_third_kernel_size, strides=1, activation="selu", kernel_initializer="lecun_normal", padding='valid'))
    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(84, activation="selu", kernel_initializer="lecun_normal"))
    model.add(keras.layers.Dense(62, activation='softmax'))

    print(model.summary())

    # Instantiate an optimizer to train the model.
    optimizer = keras.optimizers.Adam(learning_rate=wandb.config.learning_rate, beta_1=0.9, beta_2=0.999)
    # Instantiate a loss function.
    # loss_fn = keras.losses.SparseCategoricalCrossentropy(from_logits=True)


    model.compile(loss="categorical_crossentropy",
                optimizer="adam",
                metrics=["accuracy"])
    hist = model.fit(train_x, train_y, epochs=20, validation_data=(val_x, val_y), callbacks=[WandbCallback()])

In [11]:
sweep_config = {
  'method': 'random', 
  'parameters': {
      'learning_rate':{
          'values' : [0.05, 0.01, 0.005]
      },
      'n_first_node' : {
          'values' : [6, 12]
      },
      'n_second_node' : {
          'values' : [16, 32]
      },
      'n_third_node' : {
          'values' : [120, 60]
      },
      'n_first_kernel_size' : {
          'values' : [5, 3]
      },
      'n_second_kernel_size' : {
          'values' : [5, 3]
      },
      'n_third_kernel_size' : {
          'values' : [5, 3]
      }
  }
}

sweep_id = wandb.sweep(sweep_config)

wandb.agent(sweep_id, function=sweep_train, count=10)

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


Create sweep with ID: ankq1lqk
Sweep URL: https://wandb.ai/hyunseokerikjung/uncategorized/sweeps/ankq1lqk


wandb: Agent Starting Run: s00xquzi with config:
wandb: 	first_layer_node: 1000
wandb: 	learning_rate: 0.05
wandb: 	n_Dense_layer: 2
wandb: 	n_first_kernel_size: 5
wandb: 	n_first_node: 12
wandb: 	n_second_kernel_size: 5
wandb: 	n_second_node: 16
wandb: 	n_third_kernel_size: 5
wandb: 	n_third_node: 60
wandb: 	second_layer_node: 120


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 28, 28, 12)        312       
                                                                 
 average_pooling2d (AverageP  (None, 14, 14, 12)       0         
 ooling2D)                                                       
                                                                 
 conv2d_1 (Conv2D)           (None, 10, 10, 16)        4816      
                                                                 
 average_pooling2d_1 (Averag  (None, 5, 5, 16)         0         
 ePooling2D)                                                     
                                                                 
 conv2d_2 (Conv2D)           (None, 1, 1, 60)          24060     
                                                                 
 flatten (Flatten)           (None, 60)                0

wandb: WARNING The save_model argument by default saves the model in the HDF5 format that cannot save custom objects like subclassed models and custom layers. This behavior will be deprecated in a future release in favor of the SavedModel format. Meanwhile, the HDF5 model is saved as W&B files and the SavedModel as W&B Artifacts.


...

wandb: ERROR Run of12gljh errored: InternalError()
wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: eobwhmr3 with config:
wandb: 	first_layer_node: 500
wandb: 	learning_rate: 0.005
wandb: 	n_Dense_layer: 1
wandb: 	n_first_kernel_size: 3
wandb: 	n_first_node: 6
wandb: 	n_second_kernel_size: 5
wandb: 	n_second_node: 16
wandb: 	n_third_kernel_size: 5
wandb: 	n_third_node: 120
wandb: 	second_layer_node: 100


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 28, 28, 6)         60        
                                                                 
 average_pooling2d (AverageP  (None, 14, 14, 6)        0         
 ooling2D)                                                       
                                                                 
 conv2d_1 (Conv2D)           (None, 10, 10, 16)        2416      
                                                                 
 average_pooling2d_1 (Averag  (None, 5, 5, 16)         0         
 ePooling2D)                                                     
                                                                 
 conv2d_2 (Conv2D)           (None, 1, 1, 120)         48120     
                                                                 
 flatten (Flatten)           (None, 120)               0

wandb: ERROR Run eobwhmr3 errored: InternalError()
wandb: Agent Starting Run: i1yebecq with config:
wandb: 	first_layer_node: 1000
wandb: 	learning_rate: 0.05
wandb: 	n_Dense_layer: 2
wandb: 	n_first_kernel_size: 3
wandb: 	n_first_node: 6
wandb: 	n_second_kernel_size: 3
wandb: 	n_second_node: 32
wandb: 	n_third_kernel_size: 5
wandb: 	n_third_node: 60
wandb: 	second_layer_node: 120


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 28, 28, 6)         60        
                                                                 
 average_pooling2d (AverageP  (None, 14, 14, 6)        0         
 ooling2D)                                                       
                                                                 
 conv2d_1 (Conv2D)           (None, 12, 12, 32)        1760      
                                                                 
 average_pooling2d_1 (Averag  (None, 6, 6, 32)         0         
 ePooling2D)                                                     
                                                                 
 conv2d_2 (Conv2D)           (None, 2, 2, 60)          48060     
                                                                 
 flatten (Flatten)           (None, 240)               0

accuracy,▁▅▆▇▇▇▇▇▇▇██████████
epoch,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
loss,█▃▃▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁
val_accuracy,▁▅▆▅▆▆▅▆▇▆▆▆▇▇▆▇▇▇██
val_loss,█▄▃▄▂▂▂▂▁▂▂▂▂▁▃▁▂▃▂▂
accuracy,0.858
best_epoch,8
best_val_loss,0.42731
epoch,19
loss,0.38418
val_accuracy,0.85095


TESTING AND SAVING MODELS

Load and test models

In [29]:
current_test_model = keras.models.load_model("my_lenet5_emnist_model_20220515.h5")

In [30]:
current_test_model.evaluate(test_x, test_y)

3636/3636 [==============================] - 7s 2ms/step - loss: 0.3567 - accuracy: 0.8709


[0.35671404004096985, 0.8708842992782593]

Save current model

In [27]:
now = time.time()
now = time.localtime(now)
my_lenet_5_selu_model.save("my_lenet5_emnist_model_%2d%02d%02d.h5" %(now.tm_year, now.tm_mon, now.tm_mday), save_format="h5")